In [1]:
#importações basicas do script
import nltk
import re
import pandas as pd
import snscrape.modules.twitter as snsctwitter
import numpy as np 
import openpyxl


#armazenamento excel
from openpyxl import load_workbook

#importações relacionada ao SKlearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score 
from sklearn.metrics import accuracy_score 

#importações relacionadas ao nltk e vader
from nltk import word_tokenize
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.classify import NaiveBayesClassifier
from nltk import ngrams, FreqDist

#importações relacionadas ao TextBlob
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

#Tokenizador do Twitter
from sklearn.pipeline import Pipeline
from nltk.tokenize import TweetTokenizer


#from nltk.stem.snowball import SnowballStemmer
#from nltk.corpus import machado

#st = SnowballStemmer('portuguese') definindo stopwords Como portugues
stop_words = set(stopwords.words("portuguese"))

In [ ]:
#Anotações: Esse arquivo tem como objetivo um estudo em cima de coleta de uma amostra de dados do twitter sobre os presientes
#e então a analise de dados em cima desses dados coletados utilizando analisadores emocionais, lembrando que tem falhas por
#motivos linguisticos o que dificulta muito a analise dos dados coletados.

In [2]:
#Arquivo com textos traduzidos para usar como base para treinamento do MA
df = pd.read_csv('imdb-reviews-pt-br.csv', encoding='utf-8')
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [3]:
df.sentiment.value_counts()

neg    24765
pos    24694
Name: sentiment, dtype: int64

In [4]:
df.count()

id           49459
text_en      49459
text_pt      49459
sentiment    49459
dtype: int64

In [5]:
df.drop_duplicates(['text_pt'], inplace=True)
df.drop_duplicates(['text_en'], inplace=True)

In [6]:
df.count()

id           49040
text_en      49040
text_pt      49040
sentiment    49040
dtype: int64

In [7]:
Textos = df['text_pt']
classes = df['sentiment']

In [8]:
#Tratamento de dados

def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

In [9]:
Stemming('o cavaleiro anda de cavalo, o motorista de carro e o ')

'o caval and de cavalo, o motor de carr e o'

In [10]:
Limpeza_dados('testando o texto com a limpeza https://www.a.com.br :)')

'testando o texto com a limpeza  '

In [11]:
RemoveStopWords('Como gostaria de que a remocao de stopwords fosse universal para os testes')

'Como gostaria remocao stopwords universal testes'

In [64]:
#tratamento de dados
def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [stemmer.stem(i) for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))



In [13]:
#Tokenizando a frase com um tokenizador especifico de redes sociais 
frase = ' :) :-) :D ;) =D @teste testando'


texto_tokenizer = TweetTokenizer()

texto_tokenizer.tokenize(frase)

[':)', ':-)', ':D', ';)', '=D', '@teste', 'testando']

In [14]:
#Vetorizando e treinando o script
vectorizer = CountVectorizer(analyzer="word", tokenizer=texto_tokenizer.tokenize)

freq_texto = vectorizer.fit_transform(Textos)
freq_texto.shape

#49k = linhas
#154k= colunas, cada palavra é uma coluna

(49040, 154615)

In [15]:
#Passando tipo de treinamento que será feito em cima do vetor
MA = MultinomialNB()
MA.fit(freq_texto,classes)

MultinomialNB()

In [19]:
#Exemplo de como foi coletado os dados em cima de assunto e periodo entre o assunto
Presidentes = []

for i,tweet in enumerate(snsctwitter.TwitterSearchScraper(' Assunto since:2022-10-24 until:2022-10-29 lang:pt').get_items()):
    if i>100000:
        break
    Presidentes.append([tweet.date, tweet.content])
     


In [20]:
df_presidentestweets = pd.DataFrame(Presidentes, columns=['Dia', 'Texto'])
df_presidentestweetsNltk = pd.DataFrame(Presidentes, columns=['Dia', 'Texto'])
df_presidentestweetsText = pd.DataFrame(Presidentes, columns=['Dia', 'Texto'])
df_presidentestweetsNltkTraduzido = pd.DataFrame(Presidentes, columns=['Dia', 'Texto'])
df_presidentestweetsTextTraduzido = pd.DataFrame(Presidentes, columns=['Dia', 'Texto'])
TextPort = df_presidentestweets['Texto'].apply(unidecode)

In [21]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [22]:
# Removendo toda as duplicadas de tweets, mantendo só a primeira
duplicados = df_presidentestweets[df_presidentestweets.duplicated(keep='first')]
df_presidentestweets.drop_duplicates(keep='first', inplace=True) 

#separa as palavras
df_presidentestweets["TextoLimpo"] = df_presidentestweets["Texto"].str.lower().str.split()

#Tira espaço desnecessario entre palavras
df_presidentestweets["TextoLimpo"] = df_presidentestweets["TextoLimpo"].apply(lambda x: re.sub(' +', '', str(x)))

#df_presidentestweets["TextoLimpo"] = df_presidentestweets["TextoLimpo"].apply(lambda x: re.sub('[^a-zA-Z]', '', str(x)))

#Tira as palavras avulsas
pat = r'\b(?:{})\b'.format('|'.join(stop_words))
df_presidentestweets['TextoLimpo'] = df_presidentestweets['TextoLimpo'].str.replace(pat, '')
df_presidentestweets['TextoLimpo'] = df_presidentestweets['TextoLimpo'].str.replace(r'\s+', '')

df_presidentestweetsNltk['TextoLimpo'] = df_presidentestweets['TextoLimpo']
df_presidentestweetsText['TextoLimpo'] = df_presidentestweets['TextoLimpo']


In [23]:
df_presidentestweets.head(5)

,Dia,Texto,TextoLimpo
0,2022-10-28 23:59:59+00:00,O comentarista da CNN Felipe Moura Brasil anal...,"['','comentarista','','cnn','felipe','moura','..."
1,2022-10-28 23:59:59+00:00,@_Janoninho LULA INOCENTE\nBOLSONARO CRIMINOSO...,"['@_janoninho','lula','inocente','bolsonaro','..."
2,2022-10-28 23:59:59+00:00,LULA INOCENTE\nADEUS BOLSONARO\nJANONES EU AUT...,"['lula','inocente','adeus','bolsonaro','janone..."
3,2022-10-28 23:59:58+00:00,@luizatomeofic @jairbolsonaro quem é vc? pergu...,"['@luizatomeofic','@jairbolsonaro','','','vc?'..."
4,2022-10-28 23:59:58+00:00,@dramayraoficial 2️⃣2️⃣ Santa Quitéria CE com ...,"['@dramayraoficial','2️⃣2️⃣','santa','quitéria..."


In [24]:
df_presidentestweets.count()

Dia           6
Texto         6
TextoLimpo    6
dtype: int64

In [25]:
Texto = ([Preprocessing(i) for i in df_presidentestweets.Texto])
dfSk = pd.DataFrame(Texto)
dfSk = pd.DataFrame(Texto, columns = ['Texto'])
dfSk.head()

,Texto
0,comentar cnn felip mour brasil analis impact e...
1,@_janon lul inoc bolsonar crimin presid futur ...
2,lul inoc adeu bolsonar janon autoriz ving #nem...
3,@luizatomeofic @jairbolsonar vc? pergunt genuín
4,@dramayraofic 2️⃣2️⃣ sant quitér ce bolsonar 🇧...


In [26]:
freq_testes = vectorizer.transform(dfSk.Texto)

In [54]:
array = []

for t, c in zip (dfSk.Texto,MA.predict(freq_testes)):
 
    array.append([t,c])
    
    

    
    
    df_presidentestweetsSK = pd.DataFrame(array, columns = ['Texto', 'Sentimento'])
#df_presidentestweetsSK['Id'] = df_presidentestweets.Id%100000000
df_presidentestweetsSK['RedeSocial'] = 'Twitter'
df_presidentestweetsSK['Presidente'] = 'Bolsonaro'
df_presidentestweetsSK['TipoAnalise'] = 'ScriptLearn'
df_presidentestweetsSK['Periodo'] = '24-10-2022'
del df_presidentestweetsSK['Texto']




df_presidentestweetsSK.head()

,Sentimento,RedeSocial,Presidente,TipoAnalise,Periodo
0,pos,Twitter,Bolsonaro,ScriptLearn,24-10-2022
1,pos,Twitter,Bolsonaro,ScriptLearn,24-10-2022
2,neg,Twitter,Bolsonaro,ScriptLearn,24-10-2022
3,neg,Twitter,Bolsonaro,ScriptLearn,24-10-2022
4,neg,Twitter,Bolsonaro,ScriptLearn,24-10-2022


In [58]:
nomearquivo.to_excel('diretorio arquivo', index = False)

In [23]:
sid = SentimentIntensityAnalyzer()
df_presidentestweetsNltk['scores'] = df_presidentestweetsNltk['TextoLimpo'].apply(lambda review: sid.polarity_scores(review))

df_presidentestweetsNltk.head()

,Dia,Texto,TextoLimpo,scores
0,2022-10-28 23:59:59+00:00,O comentarista da CNN Felipe Moura Brasil anal...,"['','comentarista','','cnn','felipe','moura','...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,2022-10-28 23:59:59+00:00,@_Janoninho LULA INOCENTE\nBOLSONARO CRIMINOSO...,"['@_janoninho','lula','inocente','bolsonaro','...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,2022-10-28 23:59:59+00:00,LULA INOCENTE\nADEUS BOLSONARO\nJANONES EU AUT...,"['lula','inocente','adeus','bolsonaro','janone...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,2022-10-28 23:59:58+00:00,@luizatomeofic @jairbolsonaro quem é vc? pergu...,"['@luizatomeofic','@jairbolsonaro','','','vc?'...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,2022-10-28 23:59:58+00:00,@dramayraoficial 2️⃣2️⃣ Santa Quitéria CE com ...,"['@dramayraoficial','2️⃣2️⃣','santa','quitéria...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [24]:
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

sentiments_using_SIA = df_presidentestweetsNltk.Texto.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))

del df_presidentestweetsNltk['Dia']
del df_presidentestweetsNltk['Texto']
del df_presidentestweetsNltk['TextoLimpo']

df_presidentestweetsNltk['sentimento'] = sentiments_using_SIA

df_presidentestweetsNltk['RedeSocial'] = 'Twitter'
df_presidentestweetsNltk['Presidente'] = 'Lula'
df_presidentestweetsNltk['TipoAnalise'] = 'NLTK'
df_presidentestweetsNltk['Periodo'] = '24-10-2022'

df_presidentestweetsNltk.head()

,scores,sentimento,RedeSocial,Presidente,TipoAnalise,Periodo
0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",pos,Twitter,Lula,NLTK,24-10-2022
1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",pos,Twitter,Lula,NLTK,24-10-2022
2,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",pos,Twitter,Lula,NLTK,24-10-2022
3,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",pos,Twitter,Lula,NLTK,24-10-2022
4,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",pos,Twitter,Lula,NLTK,24-10-2022


In [24]:
#Testando a % de acerto do NLTK + VADER sem traduzir
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

sentiments_using_SIA = df.text_pt.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))

df['sentimentoNLTK'] = sentiments_using_SIA


acuracia = accuracy_score(df['sentiment'], df['sentimentoNLTK'])
print('A acurácia do modelo foi', acuracia)

A acurácia do modelo foi 0.531973898858075


In [25]:
def fetch_sentiment_using_textblob(text):
  
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'pos'
    elif analysis.sentiment.polarity == 0:
         return 'neut'
    else: 
        return 'neg'

sentiment_using_textblob = df_presidentestweetsText.Texto.apply(lambda tweet: fetch_sentiment_using_textblob(tweet))

del df_presidentestweetsText['Dia']
del df_presidentestweetsText['Texto']
del df_presidentestweetsText['TextoLimpo']


df_presidentestweetsText['sentimento'] = sentiment_using_textblob

df_presidentestweetsText['RedeSocial'] = 'Twitter'
df_presidentestweetsText['Presidente'] = 'Lula'
df_presidentestweetsText['TipoAnalise'] = 'TextBlob'
df_presidentestweetsText['Periodo'] = '24-10-2022'

df_presidentestweetsText.head()

,sentimento,RedeSocial,Presidente,TipoAnalise,Periodo
0,neut,Twitter,Lula,TextBlob,24-10-2022
1,neut,Twitter,Lula,TextBlob,24-10-2022
2,neut,Twitter,Lula,TextBlob,24-10-2022
3,neut,Twitter,Lula,TextBlob,24-10-2022
4,neut,Twitter,Lula,TextBlob,24-10-2022


In [26]:
#Testando a % de acerto do TextBlob Sem traduzir
def fetch_sentiment_using_textblob(text):
  
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'pos'
    elif analysis.sentiment.polarity == 0:
         return 'neut'
    else: 
        return 'neg'

sentiment_using_textblob = df.text_pt.apply(lambda tweet: fetch_sentiment_using_textblob(tweet))

df['sentimentoText'] = sentiment_using_textblob


acuracia = accuracy_score(df['sentiment'], df['sentimentoText'])
print('A acurácia do modelo foi', acuracia)

A acurácia do modelo foi 0.2917822185970636


In [71]:
from googletrans import Translator
translator = Translator()
df['text_pt'].astype(str)
df_presidentestweetsTextTraduzido['English'] = df_presidentestweetsTextTraduzido['Texto'].apply(translator.translate, src='pt', dest='en').apply(getattr, args=('text',))
df_presidentestweetsTextTraduzido

,Dia,Texto,English
0,2022-10-28 23:59:59+00:00,O comentarista da CNN Felipe Moura Brasil anal...,CNN commentator Felipe Moura Brasil analyzed t...
1,2022-10-28 23:59:59+00:00,@_Janoninho LULA INOCENTE\nBOLSONARO CRIMINOSO...,@_Janoninho Lula Innocent\nBolsonaro criminal\...
2,2022-10-28 23:59:59+00:00,"Bolsonaro já está na globo, Sérgio Moro está J...","Bolsonaro is already at Globo, Sérgio Moro is ..."
3,2022-10-28 23:59:59+00:00,Vou assistir no YouTube canal do Bolsonaro!!! ...,I'm going to watch on YouTube Bolsonaro Channe...
4,2022-10-28 23:59:59+00:00,LULA INOCENTE\nADEUS BOLSONARO\nJANONES EU AUT...,Lula Innocent\nGoodbye Bolsonaro\nJanones I au...
...,...,...,...
996,2022-10-28 23:55:27+00:00,#Eleições2022 Debate Bolsonaro na Globo ao viv...,#Elections2022 Bolsonaro Debate on Globo Live:...
997,2022-10-28 23:55:27+00:00,@nathbuenox Ainda bem q tem câmera de seguranç...,@nathbuenox I'm glad you have security camera ...
998,2022-10-28 23:55:26+00:00,"@Zambelli2210 NÃO ESQUECER \nParei na esquina,...",@Zambelli2210 Do not forget\nI stopped around ...
999,2022-10-28 23:55:26+00:00,@heytauat BOLSONARO CRIMINOSO\nfaltam 2 dias p...,@heytauat bolsonaro criminal\n2 days left for ...


In [27]:
del df_presidentestweetsTextTraduzido['Dia']

In [73]:
df_presidentestweetsTextTraduzido.to_excel(r'diretorio do arquivo', index = False)

In [36]:
df_presidentestweetsTextTraduzido.head()

,Texto
0,O comentarista da CNN Felipe Moura Brasil anal...
1,@_Janoninho LULA INOCENTE\nBOLSONARO CRIMINOSO...
2,LULA INOCENTE\nADEUS BOLSONARO\nJANONES EU AUT...
3,@luizatomeofic @jairbolsonaro quem é vc? pergu...
4,@dramayraoficial 2️⃣2️⃣ Santa Quitéria CE com ...


In [ ]:
#traduzido
def fetch_sentiment_using_textblob(text):
  
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'pos'
    elif analysis.sentiment.polarity == 0:
         return 'neut'
    else: 
        return 'neg'

sentiment_using_textblob = df_presidentestweetsTextTraduzido.English.apply(lambda tweet: fetch_sentiment_using_textblob(tweet))




df_presidentestweetsTextTraduzido['sentimento'] = sentiment_using_textblob

df_presidentestweetsTextTraduzido['RedeSocial'] = 'Twitter'
df_presidentestweetsTextTraduzido['Presidente'] = 'Bolsonaro'
df_presidentestweetsTextTraduzido['TipoAnalise'] = 'TextBlob Traduzido'
df_presidentestweetsTextTraduzido['Periodo'] = '10-10-2022'

df_presidentestweetsTextTraduzido.head()

In [51]:
df_presidentestweetsTextTraduzido.head()

,sentimento,RedeSocial,Presidente,TipoAnalise,Periodo
0,neut,Twitter,Bolsonaro,TextBlob Traduzido,03-10-2022
1,pos,Twitter,Bolsonaro,TextBlob Traduzido,03-10-2022
2,neut,Twitter,Bolsonaro,TextBlob Traduzido,03-10-2022
3,neut,Twitter,Bolsonaro,TextBlob Traduzido,03-10-2022
4,pos,Twitter,Bolsonaro,TextBlob Traduzido,03-10-2022


In [29]:
#Acuracia do TextBlob traduzido para o ingles 
def fetch_sentiment_using_textblob(text):
  
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'pos'
    elif analysis.sentiment.polarity == 0:
         return 'neut'
    else: 
        return 'neg'

sentiment_using_textblob = df.text_en.apply(lambda tweet: fetch_sentiment_using_textblob(tweet))

df['sentimentoTextTraduzido'] = sentiment_using_textblob
from sklearn.metrics import accuracy_score 

acuracia = accuracy_score(df['sentiment'], df['sentimentoTextTraduzido'])
print('A acurácia do modelo foi', acuracia)

A acurácia do modelo foi 0.6908442088091354


In [ ]:
df_presidentestweetsNltkTraduzido['English'] = df_presidentestweetsNltkTraduzido['Texto'].apply(translator.translate, src='pt', dest='en').apply(getattr, args=('text',))
df_presidentestweetsNltkTraduzido

In [ ]:
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

sentiments_using_SIA = df_presidentestweetsNltkTraduzido.English.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))


df_presidentestweetsNltkTraduzido['sentimento'] = sentiments_using_SIA

df_presidentestweetsNltkTraduzido['RedeSocial'] = 'Twitter'
df_presidentestweetsNltkTraduzido['Presidente'] = 'Lula'
df_presidentestweetsNltkTraduzido['TipoAnalise'] = 'NLTK Traduzido'
df_presidentestweetsNltkTraduzido['Periodo'] = '03-10-2022'



In [ ]:
del df_presidentestweetsNltkTraduzido['Dia']
del df_presidentestweetsNltkTraduzido['Texto']
del df_presidentestweetsNltkTraduzido['English']


In [ ]:
df_presidentestweetsNltkTraduzido.head()

In [30]:
#Testando a % de acerto do NLTK + VADER traduzido para o ingles
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

sentiments_using_SIA = df.text_en.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))

df['sentimentoNLTKTraduzido'] = sentiments_using_SIA


acuracia = accuracy_score(df['sentiment'], df['sentimentoNLTKTraduzido'])
print('A acurácia do modelo foi', acuracia)

A acurácia do modelo foi 0.692536704730832


In [ ]:
df_presidentestweets.head()

In [ ]:
df_presidentestweetsSK.count()

In [ ]:
pd.DataFrame(df_presidentestweetsSK.Sentimento.value_counts())
pd.DataFrame(df_presidentestweetsSK.Sentimento.value_counts()).plot(kind='bar')


In [51]:
df_presidentestweetsSK.to_excel(r'diretorio arquivo', index = False)

In [ ]:
df_presidentestweetsSK.count()

In [ ]:
df_presidentestweetsNltk.count()

In [ ]:
df_presidentestweetsText.count()

In [31]:
import numpy as np
import pandas as pd

my_array = np.array([[t,c]])

df = pd.DataFrame(my_array, columns = ['Column_A','Column_B'])

print(df)
print(type(df))

                                Column_A Column_B
0  @brom_elis @jairbolsonar perfil dela?      neg
<class 'pandas.core.frame.DataFrame'>


In [32]:
from sklearn.pipeline import Pipeline

pipeline_simples = Pipeline([
  ('counts', CountVectorizer()),
  ('classifier', MultinomialNB())
])

pipeline_simples.fit(Textos,classes)
pipeline_simples.steps
resultados = cross_val_predict(pipeline_simples, Textos, classes, cv=10)
def Metricas(Ma, Textos, classes):
  resultados = cross_val_predict(Ma, Textos, classes, cv=10)
  return 'Acurácia do modelo: {}'.format(metrics.accuracy_score(classes,resultados))

Metricas(pipeline_simples,Textos,classes)

'Acurácia do modelo: 0.7999796084828711'

In [33]:
print (MA.classes_)
MA.predict_proba(freq_testes).round(2)

['neg' 'pos']


array([[0.01, 0.99],
       [0.29, 0.71],
       [0.53, 0.47],
       ...,
       [0.1 , 0.9 ],
       [0.5 , 0.5 ],
       [0.63, 0.37]])

In [34]:
df_presidentestweetsNltk.head()

,sentimento,RedeSocial,Presidente,TipoAnalise,Periodo
0,pos,Twitter,Lula,NLTK,24-10-2022
1,pos,Twitter,Lula,NLTK,24-10-2022
2,pos,Twitter,Lula,NLTK,24-10-2022
3,pos,Twitter,Lula,NLTK,24-10-2022
4,pos,Twitter,Lula,NLTK,24-10-2022


In [35]:
df_presidentestweetsText.count()

sentimento     1001
RedeSocial     1001
Presidente     1001
TipoAnalise    1001
Periodo        1001
dtype: int64

In [39]:
#Adiciona ANALISE NLTK ao EXCEL

book = load_workbook(r'diretorio arquivo')
writer = pd.ExcelWriter(r'diretorio arquivo', engine='openpyxl')
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}

for sheetname in writer.sheets:
    df_presidentestweetsNltk.to_excel(writer,sheet_name='Sheet1', startrow=writer.sheets['Sheet1'].max_row, index = False,header= False)

writer.save()

In [40]:
#Adiciona ANALISE TextBlob ao EXCEL

book = load_workbook(r'diretorio arquivo')
writer = pd.ExcelWriter(r'diretorio arquivo', engine='openpyxl')
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}

for sheetname in writer.sheets:
    df_presidentestweetsText.to_excel(writer,sheet_name='Sheet1', startrow=writer.sheets['Sheet1'].max_row, index = False,header= False)

writer.save()

In [ ]:
book = load_workbook(r'diretorio arquivo')
writer = pd.ExcelWriter(r'diretorio arquivo', engine='openpyxl')
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}

for sheetname in writer.sheets:
    df_presidentestweetsTextTraduzido.to_excel(writer,sheet_name='Sheet1', startrow=writer.sheets['Sheet1'].max_row, index = False,header= False)

writer.save()

In [2]:
from googletrans import Translator
translator = Translator()

teste = []

for i,tweet in enumerate(snsctwitter.TwitterSearchScraper(' Lula since:2022-10-03 until:2022-10-09 lang:pt').get_items()):
    if i>1000:
        break
    teste.append([tweet.date, tweet.content])
     
dfc = pd.DataFrame(teste, columns=['Dia', 'Texto'])


# Removendo toda as duplicadas de tweets, mantendo só a primeira
duplicados = dfc[dfc.duplicated(keep='first')]
dfc.drop_duplicates(keep='first', inplace=True) 




dfc['English'] = dfc['Texto'].apply(translator.translate, src='pt', dest='en').apply(getattr, args=('text',))
dfc


KeyboardInterrupt



In [61]:
dfc = pd.read_csv('arquivo', encoding='utf-8')
dfc.head()

,Unnamed: 0,Dia,Texto,English
0,0,2022-10-08 23:59:59+00:00,@BolsonaroSP @jairbolsonaro Mama no estado e d...,@Bolsonarosp @jairbolsonaro breasts in the sta...
1,1,2022-10-08 23:59:59+00:00,Críticos de Bolsonaro (PL) usaram as redes soc...,Bolsonaro (PL) critics used social networks to...
2,2,2022-10-08 23:59:58+00:00,@PaparazzoRN @jairbolsonaro @futbolaco_vasco @...,@Paparazzorn @jairbolsonaro @futbolaco_vasco @...
3,3,2022-10-08 23:59:58+00:00,@Sayobernardino @paulosdealmeida @VEJA É o mes...,@Sayobernardino @pauliulosdealmeida @jeir is t...
4,4,2022-10-08 23:59:58+00:00,@eumesmavivi_ Gostei desse cara do áudio. Sou ...,@eumesmavivi_ I liked this audio guy.I'm Bolso...


In [62]:
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

sentiments_using_SIA = dfc.English.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))


dfc['sentimento'] = sentiments_using_SIA

dfc['RedeSocial'] = 'Twitter'
dfc['Presidente'] = 'Bolsonaro'
dfc['TipoAnalise'] = 'NLTK Traduzido'
dfc['Periodo'] = '03-10-2022'

dfc.head()

,Unnamed: 0,Dia,Texto,English,sentimento,RedeSocial,Presidente,TipoAnalise,Periodo
0,0,2022-10-08 23:59:59+00:00,@BolsonaroSP @jairbolsonaro Mama no estado e d...,@Bolsonarosp @jairbolsonaro breasts in the sta...,pos,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022
1,1,2022-10-08 23:59:59+00:00,Críticos de Bolsonaro (PL) usaram as redes soc...,Bolsonaro (PL) critics used social networks to...,neg,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022
2,2,2022-10-08 23:59:58+00:00,@PaparazzoRN @jairbolsonaro @futbolaco_vasco @...,@Paparazzorn @jairbolsonaro @futbolaco_vasco @...,pos,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022
3,3,2022-10-08 23:59:58+00:00,@Sayobernardino @paulosdealmeida @VEJA É o mes...,@Sayobernardino @pauliulosdealmeida @jeir is t...,neg,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022
4,4,2022-10-08 23:59:58+00:00,@eumesmavivi_ Gostei desse cara do áudio. Sou ...,@eumesmavivi_ I liked this audio guy.I'm Bolso...,pos,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022


In [63]:
del dfc['Unnamed: 0'] 
del dfc['Dia'] 
del dfc['Texto'] 
del dfc['English'] 

In [64]:
dfc.head()

,sentimento,RedeSocial,Presidente,TipoAnalise,Periodo
0,pos,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022
1,neg,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022
2,pos,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022
3,neg,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022
4,pos,Twitter,Bolsonaro,NLTK Traduzido,03-10-2022


In [65]:
dfc.to_excel(r'diretorio arquivo', index = False)

In [118]:
#testando classificador de acordo com o que foi estudado para ver se o resultado é coerente

exemplo = ['a experiencia foi otima, mas o filme é bem ruim',
          'O governo é otimo',
          'o governo é ruim',
          'odeio o governo',
          'A estrutura do governo atual é normal']

In [119]:
exemplo = [Limpeza_dados(i) for i in testes]
exemplo = [RemoveStopWords(i) for i in testes]

In [120]:
freq_testes = vectorizer.transform(exemplo)

In [121]:
for t, c in zip (testes,MA.predict(freq_testes)):
    print (t +", "+ c)

experiencia otima, filme bem ruim, neg
governo otimo, pos
governo ruim, neg
odeio governo, neg
estrutura governo atual normal, pos


In [122]:
print (MA.classes_)
MA.predict_proba(freq_testes).round(3)

['neg' 'pos']


array([[0.819, 0.181],
       [0.34 , 0.66 ],
       [0.8  , 0.2  ],
       [0.65 , 0.35 ],
       [0.257, 0.743]])